+++
title = "Multiple correspondence analysis"
menu = "main"
weight = 3
toc = true
aliases = ["mca"]
+++

## Resources

- [Computation of Multiple Correspondence Analysis, with code in R](https://core.ac.uk/download/pdf/6591520.pdf)

## Data

Multiple correspondence analysis is an extension of correspondence analysis. It should be used when you have more than two categorical variables. The idea is to one-hot encode a dataset, before applying correspondence analysis to it.

As an example, we're going to use the [balloons dataset](https://archive.ics.uci.edu/ml/machine-learning-databases/balloons/) taken from the [UCI datasets website](https://archive.ics.uci.edu/ml/datasets.html).

In [1]:
import pandas as pd

dataset = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/balloons/adult+stretch.data')
dataset.columns = ['Color', 'Size', 'Action', 'Age', 'Inflated']
dataset.head()

,Color,Size,Action,Age,Inflated
0,YELLOW,SMALL,STRETCH,ADULT,T
1,YELLOW,SMALL,STRETCH,CHILD,F
2,YELLOW,SMALL,DIP,ADULT,F
3,YELLOW,SMALL,DIP,CHILD,F
4,YELLOW,LARGE,STRETCH,ADULT,T


## Fitting

In [13]:
import prince

mca = prince.MCA(
    n_components=3,
    n_iter=3,
    copy=True,
    check_input=True,
    engine='sklearn',
    random_state=42,
    one_hot=True,
    get_dummies=True
)
mca = mca.fit(dataset)

(19, 5)


The way MCA works is that it one-hot encodes the dataset, and then fits a correspondence analysis. In case your dataset is already one-hot encoded, you can specify `one_hot=False` to skip this step.

In [14]:
one_hot = pd.get_dummies(dataset)

mca_no_one_hot = prince.MCA(one_hot=False)
mca_no_one_hot = mca_no_one_hot.fit(one_hot)

(19, 10)


## Eigenvalues

In [15]:
mca.eigenvalues_summary

,eigenvalue,% of variance,% of variance (cumulative)
component,,,
0,0.402,40.17%,40.17%
1,0.211,21.11%,61.28%
2,0.186,18.56%,79.84%


## Coordinates

In [16]:
mca.row_coordinates(dataset).head()

,0,1,2
0,0.705387,5.369158e-15,0.758639
1,-0.386586,5.724889e-15,0.626063
2,-0.386586,4.807799e-15,0.626063
3,-0.852014,5.108782e-15,0.562447
4,0.783539,-6.333333e-01,0.130201


In [17]:
mca.column_coordinates(dataset).head()

,0,1,2
Color_PURPLE,0.117308,6.892024e-01,-0.641270
Color_YELLOW,-0.130342,-7.657805e-01,0.712523
Size_LARGE,0.117308,-6.892024e-01,-0.641270
Size_SMALL,-0.130342,7.657805e-01,0.712523
Action_DIP,-0.853864,-6.209900e-16,-0.079340


## Visualization

In [18]:
mca.plot(
    dataset,
    x_component=0,
    y_component=1,
    show_column_markers=True,
    show_row_markers=True,
    show_column_labels=False,
    show_row_labels=False
)

/opt/anaconda3/envs/nibm_ml/lib/python3.10/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/envs/nibm_ml/lib/python3.10/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/envs/nibm_ml/lib/python3.10/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/envs

alt.LayerChart(...)

## Contributions

In [19]:
mca.row_contributions_.head().style.format('{:.0%}')

,0,1,2
0,7%,0%,16%
1,2%,0%,11%
2,2%,0%,11%
3,10%,0%,9%
4,8%,10%,0%


In [20]:
mca.column_contributions_.head().style.format('{:.0%}')

,0,1,2
Color_PURPLE,0%,24%,23%
Color_YELLOW,0%,26%,26%
Size_LARGE,0%,24%,23%
Size_SMALL,0%,26%,26%
Action_DIP,15%,0%,0%


## Cosine similarities

In [21]:
mca.row_cosine_similarities(dataset).head()

,0,1,2
0,0.461478,2.673675e-29,0.533786
1,0.152256,3.338988e-29,0.399316
2,0.152256,2.354904e-29,0.399316
3,0.653335,2.348969e-29,0.284712
4,0.592606,3.871772e-01,0.016363


In [22]:
mca.column_cosine_similarities(dataset).head()

,0,1,2
Color_PURPLE,0.015290,5.277778e-01,0.456920
Color_YELLOW,0.015290,5.277778e-01,0.456920
Size_LARGE,0.015290,5.277778e-01,0.456920
Size_SMALL,0.015290,5.277778e-01,0.456920
Action_DIP,0.530243,2.804572e-31,0.004578
